### Kreiranje održive strukture

- model podataka koji će omogućiti kreiranje termin za raspored
- **model realizacije**

### Parsiranje zauzeća predavača

- na osnovu ```Rasporeda (Ponedeljak.xlsx, Utorak.xlsx,...)```
- prebrojavamo koliko termina ima svaki predavač
    - nedeljno zauzeće - ukupan broj časova koje drži

In [14]:
from model_parser import *
from realizacija_linked_model import *
from collections import defaultdict

In [15]:
raspored = RasporedTermin.read_list_from_file('4_raspored_spojen', dir_path='../out_data_2023_letnji/')
raspored_izmene = RasporedTermin.read_list_from_file('4_raspored_spojen', dir_path='../out_data_2023_letnji/')

schedule = MeetingSchedule.read_entity_from_file('4_svi_fajlovi_spojeni', dir_path='../out_data_2023_letnji/')
schedule_izmene = MeetingSchedule.read_entity_from_file('4_svi_fajlovi_spojeni', dir_path='../out_data_2023_letnji/')

In [16]:
def count_unknown_termini(
        raspored: list[RasporedTermin]
) -> list[RasporedTermin]:
    # 41 termin nemaju pronadjen predmet i studijski program za koji su vezani
    unknown = [x for x in raspored if x.studProgram_id == None and x.predmet_id == None]
    return unknown

In [17]:
def remove_unkown_termini(
        raspored: list[RasporedTermin]
) -> list[RasporedTermin]:
    # izbacujemo 41 termin
    return [x for x in raspored if x.studProgram_id != None and x.predmet_id != None]

In [18]:
def count_asistent_zauzeca(
        raspored: list[RasporedTermin]
) -> dict[tuple[str, str], dict[str, int]]:
    # povratna struktura: mapira id studentskog programa i predmeta na mapu svih asistenata koji drze taj predmet
    # {
    #     (stud_program_id, predmet_id): {
    #         asistent_id_1: 4,
    #         asistent_id_2: 3,
    #         asistent_id_3: 1
    #     },
    #     (stud_program_id, predmet_id_1): {...}
    # }
    zauzeca = defaultdict(lambda: defaultdict(lambda: 0))
    for termin in raspored:
        # brojimo samo vezbe, racunamo da predavanja drzi jedan profesor sva
        if 'vežbe' not in termin.tipNastave:
            continue
        zauzeca[(termin.studProgram_id, termin.predmet_id)][termin.predavac_id] += 1
    return zauzeca


In [19]:
def create_predmet_predavac(
        termin: RasporedTermin,
        zauzeca: dict[tuple[str, str], dict[str, int]],
        stud_program_id: str,
        predmet: Predmet,
        predavaci: list[Predavac]
) -> PredmetPredavac:
    # profesor
    profesor = next(x for x in predavaci if x.id == termin.predavac_id)
    # ostali profesori
    ostali_profesori = []
    if termin.ostaliPredavaci:
        for prof_id in termin.ostaliPredavaci:
            prof = next(x for x in predavaci if x.id == prof_id)
            ostali_profesori.append(prof.id)
    # asistenti
    # nadji sve iz seta koji imaju taj studijski program i taj predmet
    asistenti_zauzeca = zauzeca[(stud_program_id, predmet.id)]
    asistenti = []
    for asistent_id, broj_termina in asistenti_zauzeca.items():
        asistenti.append(AsistentZauzeca(asistent_id, broj_termina))
    predmet_predavac = PredmetPredavac(predmet.id, predmet.oznaka, predmet.godina, predmet.plan, profesor.id, ostali_profesori, asistenti)
    return predmet_predavac

In [20]:
def create_studijski_program_predmeti(
        realizacija: Realizacija,
        stud_program_id: str
) -> StudijskiProgramPredmeti:
    # ako postoji studijski program u okviru realizacije, dobavi ga
    studijski_program_predmeti = next((x for x in realizacija.studijskiProgramPredmeti if x.studijskiProgramId == stud_program_id), None)
    # ako ne postoji, kreiraj novi sa praznom listom predmet-predavaca
    if studijski_program_predmeti is None:
        studijski_program_predmeti = StudijskiProgramPredmeti(stud_program_id, stud_program_id, [])
        # dodavanje novog studijskog program u realizaciju samo ako vec ne postoji
        realizacija.studijskiProgramPredmeti.append(studijski_program_predmeti)
    return studijski_program_predmeti

In [21]:
def create_realizacija(
        godina: str,
        semestar: str,
        raspored: list[RasporedTermin],
        zauzeca: dict[tuple[str, str], dict[str, int]],
        stud_programi: list[StudijskiProgram],
        predmeti: list[Predmet],
        predavaci: list[Predavac]
) -> Realizacija:
    # kreiranje prazne realizacije
    realizacija = Realizacija(godina, semestar, [])

    # studProgram_id + ' ' + predmet_id
    seen_predmeti = set()

    for termin in raspored:
        # sastavljamo listu svih predmeta na studijskom programu u realizaciji
        # pronalazimo predmete po predavanjima
        if termin.tipNastave != 'Pred.':
            continue
        # ako smo vec videli taj predmet na tom studijskom programu, ne dodajemo ga
        if (termin.studProgram_id + ' ' + termin.predmet_id) in seen_predmeti:
            continue

        # studijski program
        stud_program = next(x for x in stud_programi if x.id == termin.studProgram_id)
        # predmet
        predmet = next(x for x in predmeti if x.id == termin.predmet_id)

        predmet_predavac = create_predmet_predavac(termin, zauzeca, stud_program.id, predmet, predavaci)
        studijski_program_predmeti = create_studijski_program_predmeti(realizacija, stud_program.id)
        studijski_program_predmeti.predmetPredavaci.append(predmet_predavac)

        # dodavanje vidjenog para stud_program-predmet (da ne bismo dodali vise puta)
        seen_predmeti.add(stud_program.id + ' ' + predmet.id)
    
    return realizacija

#### Dodavanje spojenih predmeta u realizaciju

In [22]:
def find_joined_termini(
    raspored: list[RasporedTermin]
) -> dict[str, list[RasporedTermin]]:
    # { predmet_id: [termin1, termin2] }
    joined_termini = defaultdict(lambda: [])
    for termin in raspored:
        ukupno_studenata = termin.ukupnoStud.lower()
        if 'd' in ukupno_studenata and termin.tipNastave == 'Pred.':
            joined_termini[termin.predmet_id].append(termin)
    return joined_termini

In [23]:
def add_joined_predmeti(
        realizacija: Realizacija,
        raspored: list[RasporedTermin],
        zauzeca: dict[tuple[str, str], dict[str, int]],
        predmeti: list[Predmet],
        predavaci: list[Predavac]
) -> Realizacija:
    joined_termini = find_joined_termini(raspored)
    # za svaki spojeni predmet
    for predmet_id, termini in joined_termini.items():
        # pronaci glavni predmet (po idu) i sporedne spojene po oznaci predmeta
        predmet_glavni = next(x for x in predmeti if x.id == predmet_id)
        predmeti_ostali = [x for x in predmeti if x.oznaka == predmet_glavni.oznaka and x.plan == predmet_glavni.plan and x.id != predmet_glavni.id]
        # proci kroz sve termine koji su spojeni
        termin = termini[0]
        for predmet in predmeti_ostali:
            stud_program_predmeti = create_studijski_program_predmeti(realizacija, predmet.studijskiProgram)
            existing_predmet = next((x for x in stud_program_predmeti.predmetPredavaci if x.predmetId == predmet.id), None)
            if not existing_predmet:
                new_predmet = create_predmet_predavac(termin, zauzeca, predmet.studijskiProgram, predmet, predavaci)
                stud_program_predmeti.predmetPredavaci.append(new_predmet)
    return realizacija

In [24]:
raspored_izmene = remove_unkown_termini(raspored_izmene)
zauzeca = count_asistent_zauzeca(raspored_izmene)
realizacija = create_realizacija('2023', 'Z', raspored_izmene, zauzeca, schedule_izmene.studProgramList, schedule_izmene.predmetList, schedule_izmene.predavacList)
realizacija = add_joined_predmeti(realizacija, raspored_izmene, zauzeca, schedule_izmene.predmetList, schedule_izmene.predavacList)
ReadWrite.write_to_file(realizacija, '5_realizacija', dir_path='../out_data_2023_letnji/')